## Solução para o jogo

Modelo de Dou et al., 2020.



In [1]:
#para mostrar todos os resultados e não apenas o último
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
#libraries
import numpy as np
from numba import jit, jitclass, njit, float64,  int32, char #para otimizar as funções
import numba as nb
import matplotlib.pyplot as plt
%matplotlib inline
import quantecon as qe
from scipy.stats import beta

from random import uniform #para a draw da uniforme(0,1)
import math

import time #tempo de computação


### Passo 0: definindo comandos do latex para facilitar a escrita

Aqui também podemos descrever as funções do jogo

$%conjunto de informações de s em t$
$\newcommand{\Is}[1]{\theta_{s{#1}}, l_{s{#1}}, l_{j{#1}}}$
$%macros para facilitar a escrita de funções valor$


$%conjunto de informações de j em t$
$\newcommand{\Ij}[1]{\theta_{j{#1}}, l_{s{#1}}, l_{j{#1}}}$

$%função valor de s em t$
$\newcommand{\Ws}[1]{ W_{s{#1}} ( \Is{{#1}}) }$

$%função valor de j em t$
$\newcommand{\Wj}[1]{ W_{j{#1}} ( \Ij{{#1}}) }$

$%operador esperança de s em t. 2 argumentos: o primeiro é o período e o segundo é o termo que ela tira a esperança$

$\newcommand{\Es}[2]{\mathbb{E_{#1}^{s} \big[ {#2}  \mid ( \Is{#1} )    \big] }}$

$%minúsculo não mostra o conjunto de informação$
$\newcommand{\es}[2]{\mathbb{E_{#1}^{s} \big[ {#2}  \big] }}$

$%minúsculo não mostra o conjunto de informação$
$%final b de big para aumentar os parênteses$
$\newcommand{\esb}[2]{\mathbb{E_{#1}^{s} \bigg[ {#2}  \bigg] }}$

$%operador esperança de j em t. 2 argumentos: o primeiro é o período e o segundo é o termo que ela tira a esperança$

$\newcommand{\Ej}[2]{\mathbb{E_{#1}^{j} \big[ {#2}  \mid ( \Ij{#1} )    \big] }}$

$%minúsculo não mostra o conjunto de informação$
$\newcommand{\ej}[2]{\mathbb{E_{#1}^{j} \big[ {#2}  \big] }}$

$%minúsculo não mostra o conjunto de informação$
$%final b para aumentar os parênteses$
$\newcommand{\ejb}[2]{\mathbb{E_{#1}^{j} \bigg[ {#2}  \bigg] }}$


$%comando para usar o máximo com chaves grandes$
$\newcommand{\maximo}[1]{\max \bigg\{ #1 \bigg\}}$


In [3]:
#parâmetros do jogo, apenas para ilustração a princípio
#coloquei os mesmos parãmetros dos autores (Tabela 4). No caso dos dados, usei as médias (Panel B)





μ = 4.566 #número de meses entre períodos, não entendi onde entra ainda
ρ = 0.884 # (1 - ρ) é a taxa de depreciação da empresa a cada período
β = 9.84 #usamos aqui a distribuição Uniforme, depois vamos mudar isto
c0 = 0.044 #custo fixo de ir para a corte
c1 = 0.015 #custo variável de ir para a corte


θ_s0 = 0.28 #habilidade inicial de s
θ_j0 = 0.36 #habilidade inicial de j

In [4]:
#informações que virão dos dados

#valores médios de L, Dj e Ds. Arredondei para duas casas decimais ao simular o modelo
# 0.2493718592964824
# 0.35113065326633164
# 0.27701005025125625



λj = 0.346 #probabilidade de j propor a cada turno. Não precisaremos estimar, isso virá dos dados

Vmax = 1
L = 0.25
# L_s = 10
# L_j = 10

#valores da dívida de cada jogador (virá dos dados, aqui é exemplo):
Dj = 0.35
Ds = 0.28

### Passo 1: número máximo de turnos

Calculado com base nos parâmetros


t é tal que

$$ \rho^{t-1} V_{max} = L$$


$$ t = \frac{log(L) - log(V_{max})} {log(\rho)} +1 $$

In [5]:
#função para calcular o máximo de turnos do jogo
def maximo_de_turnos(ρ, Vmax, L):
    
    T = (math.log(L) - math.log(Vmax))/math.log(ρ) + 1
    
    #arredonda para baixo porque queremos o último período no qual o valor de continuação é maior ou igual ao de liquidação
    T = math.floor(T)
        
    return T
    

In [6]:
T = maximo_de_turnos(ρ, Vmax, L)

T

12

In [7]:
#valor máximo de reorganização da firma a cada período


#sequência de valores da firma para cada período
#tem que deixar como dtype = np.float para pegar valores decimais

def Vt(Vmax, T, ρ):
    
    

    V = np.empty(T, dtype=np.float)

    for t in range(T):
        
        #no período t = 0, ela é o Vmax. E no período t = 1, também, pois não depreciou ainda

        if(t == 0):
            V[t] = Vmax

        else:
            V[t] = ρ**(t-1) * Vmax
            
    return V

V = Vt(Vmax, T, ρ)

### Passo 2: Definir os arrays para guardar as funções valor de cada período

São:

* 100 slots para habilidade do jogador

* 100 slots para o lower bound do adversário

* 100 slots para o lower bound do próprio jogador no próximo período

* 2 slots para dizer se o jogador está propondo ou não
    
* T slots para marcar o período da função valor



In [9]:
#slots para cada habilidade
grid_size = 100


#a vantagem de colocar os dados assim é que se eu quiser teta_s = 0.115, basta procurar θs_vals[114]
θs_vals = np.linspace(0.01, 1, grid_size) 
θj_vals  = np.linspace(0.01, 1, grid_size)  


#vetores dos lower bounds são similares aos das habilidades
ℓs_vals = θs_vals
ℓj_vals = θj_vals


#teste
# θs_vals[99 - 1]

#como deixar todos os valores com apenas duas casas decimais? Alguns valores ficam esquisitos



In [53]:
#valor de liquidação


#a dívida total é sempre a soma das dívidas
D = Ds + Dj

#o custo total é uma função do tempo

@jit
def C(t):
    if(t == 0):
        return 0
    else:
        Ct = c0 * D + c1 * t * D

        return Ct

#e os valores de liquidação também são função do tempo

    


In [127]:
#funções para achar os valores nas matrizes
@jit
def find(y):
    
    x = 100*y - 1
    
    #transformando em int para usar como índice nas matrizes
    
    x = int(x)
    
    return x


### Passo 3: fazer o cálculo das funções valor em T-1, T-2, ..., 1.

### Função para tirar um draw da distribuição Beta


Vamos usar o método da amostragem da inversa da CDF (https://en.wikipedia.org/wiki/Inverse_transform_sampling_method). Outra referência que usei foi: https://blogs.sas.com/content/iml/2013/07/22/the-inverse-cdf-method.html#:~:text=The%20exponential%20distribution%20has%20probability,log(1%E2%80%93u).

A CDF da Beta é 

$$ F_{\beta} ( \theta_{t+1} \mid \theta_{t} ) = 1 - \frac{ (1 - \theta_{t+1})^\beta}{ (1 - \theta_{t})^\beta }, \, \, \theta_{t} \leq \theta_{t+1} \leq 1, \, \beta \geq 1$$

Para invertê-la, basta procurarmos o valor de x tal que $F(x) = u$, onde u é uma retirada da distribuição Uniforme(0,1).

Fazendo os cálculos, esse valor de x é (ou $\theta_{t+1}$, no caso)


$$ \theta_{t+1} =  1 - exp \bigg\{ \frac{1}{\beta} \big[  log (1 - u) + \beta * log(1 - \theta_{t}) \big] \bigg\} $$


In [55]:
#código para tirar draw da distribuição beta
@njit
def draw_beta(info_hoje):
    
    
    #se for igual a 1, retorna 1. Não usei a fórmula da inversa CDF porque teríamos log (0)
    if(info_hoje == 1):
        return 1
    else:
    
        u = uniform(0, 1)
        x = 1 - math.exp( (1/β) * (math.log(1-u) + β*math.log(1-info_hoje)) )

        return x




# testando

# draw_beta(0.5)



# #teste com draw da função UNIFORME
# def draw_beta(info_hoje):
#     u = uniform(info_hoje, 1)
#     return u


# # testando
# draw_beta(0.99)

### Função para tirar o valor esperado do teta amanhã, dada a informação hoje.

Usamos a forma fechada, baseada na esperança de uma variável aleatória truncada: https://en.wikipedia.org/wiki/Truncated_distribution



$$ \mathbb{E} \big[ X \mid X > y \big] =  \frac{\int_{y}^{\infty}  x g(x) dx} {1 - F(y)} $$


Onde:

* $f(x)$ é a pdf da Beta, sem truncar. No caso, usamos a = 1 e b = $\beta$
* $ g(x) = f(x)$ sempre que $x > y$ e 0 caso contrário
* $F(y)$ é a CDF da Beta, sem truncar, avaliada em y


Em nosso caso, y será a informação que temos hoje para formar a expectativa sobre a habilidade amanhã. Na prática, y será ou a habilidade do credor no período atual $\theta_{k,t}, k \in \{s,j\}$ ou o lower bound da habilidade do credor adversário no período atual $l_{k,t}$.

Ilustraremos a fórmula usando $l_{t}$:

<!-- 
$$ \mathbb{E} \big[ \theta_{t+1} \mid \theta_{t+1} > l_t \big] =  \frac{ l_t (1 - l_t)^\beta + \frac{(1-l_t)^{\beta+1} }{(\beta+1)} } {1 - (1 - l_t)^\beta} $$ -->



$$ \mathbb{E} \big[ \theta_{t+1} \mid \theta_{t+1} > l_t \big] =  \frac{ l_t (1 - l_t)^\beta + (1-l_t)^{\beta+1}(\beta+1)^{-1} } {(1 - l_t)^{\beta} } $$



In [56]:
@njit
def expec_beta(info_hoje):
    
    #retorna 1 se a info_hoje for 1. A CDF não suporta 1
    if info_hoje == 1:
        return 1
    else:
        num = info_hoje * (1 - info_hoje)**β + ((1-info_hoje)**(β+1))/(β+1)

        denom = (1-info_hoje)**(β)

        return round(num/denom,2)
    

# testando, com beta = 1 e lt = 0.5, deve achar 0.75


# expec_beta(0.5)


# #para os outros valores, não pode ser superior a 1. E tem que ser crescente
# expec_beta(0.3)

# expec_beta(0.99)
# expec_beta(1)

In [57]:
#binning da pdf beta

#gerando 1000 draws

def bin(info_hoje, ndraws):
    
    beta_vals = []

    for k in range(ndraws):
        beta_amanha = draw_beta(info_hoje)

        beta_vals.append(beta_amanha)
    
    #cria os bins e conta quantos valores estão dentro deles
    teta_bins = np.zeros(len(θj_vals))  
    
    for t in range(len(teta_bins)):
        
        #ajustando os bins iniciais e final
        if(t==0):
            pre = 0
        else:
            pre = (θj_vals[t-1]+θj_vals[t])/2
               
        
        if(t==len(teta_bins)-1):
            pos = 1
        else: 
            pos = (θj_vals[t]+θj_vals[t+1])/2
        
        
        for b in beta_vals:
            
            if(b >= pre and b <= pos):
                
                teta_bins[t] += 1
    
    return teta_bins/ndraws
        
        
        
        
#às vezes soma um pouco a mais que um (tipo a sétima casa decimal fica maior que zero, mas ok)
# sum(bin(0.9,1000))




In [58]:
#vamos gerar uma matriz com 100 linhas e 100 colunas
#cada coluna vai representar as probabilidades de teta_amanhã dado teta hoje
#a linha 1 significa que teta_hoje é 0.01
#assim, a linha 1 tem as probabilidades de teta_amanhã dado que teta_hoje é 0.01




#probability mass function
pmf = np.zeros((100,100))

#exemplo para ilustrar
# pmf[0,] = bin(θj_vals[0], β, 1000)


#populando a pmf:

for t in range(len(θj_vals)):
    
    pmf[t,:] = bin(θj_vals[t], 1000)

#quais as probabilidades de teta_amanhã se eu sei que teta_hoje = 0.5?

# sum(pmf[53,:])

In [59]:
#teste eliminando valores abaixo de teta_hoje


pmf[54,:]

pmf[54, 53:100]

sum(pmf[54, 54:100])

pmf[99,99:100]

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.129, 0.173, 0.136, 0.128, 0.098, 0.085, 0.066, 0.055, 0.035,
       0.025, 0.022, 0.008, 0.01 , 0.005, 0.008, 0.004, 0.003, 0.004,
       0.   , 0.   , 0.001, 0.004, 0.001, 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   ])

array([0.   , 0.129, 0.173, 0.136, 0.128, 0.098, 0.085, 0.066, 0.055,
       0.035, 0.025, 0.022, 0.008, 0.01 , 0.005, 0.008, 0.004, 0.003,
       0.004, 0.   , 0.   , 0.001, 0.004, 0.001, 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   ])

1.0

array([1.])

### Como obter o valor esperado da função valor no período seguinte?

Multiplicando a coluna correta de pmf pela coluna correta de Ws_array


Agora que temos a pdf, podemos fazer assim:


$$ \es{t}{ \Ws{t+1} } = \sum_{\theta_{t+1}=0.01}^{1.00} prob(\Is{t+1}) * \Ws{t+1}$$


Na prática, já saberemos $l_{s,t+1}$ e $l_{j,t+1}$, então poderemos fixar esses valores


$$ \es{t}{ \Ws{t+1} } = \sum_{\theta_{t+1}=0.01}^{1.00} prob(\theta_{t+1}) * [ \Ws{t+1} \mid l_{s,t+1}, l_{j,t+1} ]$$


Vamos fazer essa soma usando multiplicação de vetores:

* aproveitaremos que dá pra saber os lower bounds do período seguinte a cada situação, então a única incerteza é sobre teta
* multiplicaremos pmf (vetor linha)
* pelo vetor coluna W correspondente


Como achar a entrada correspondente na matriz?
Por exemplo, achar qual entrada corresponde a lst =  0.58. Basta escrever 57 na entrada correspondente. Ou usar a função find(0.58).




### Passando o código para o formato Class

https://stackoverflow.com/questions/3521715/call-a-python-method-by-name


In [60]:
def call_method(o, name):
    return getattr(o, name)


In [84]:
#Bellman equations for both players k and his opponent, player m
#only works if we call the creditors "s" or "j" when assigning the Class

# Bellman_data = [
#     ('creditor', nb.types.string), #creditor is as string
#     ('opponent', nb.types.string), #opponent is also a string
#     ('k', nb.types.string), #k is as string
#     ('m', nb.types.string), #m is also a string
#     ('t', int32), #period
#     ('θkt', float64), #real hability of player k
#     ('θk_previous', float64), #real hability of player k in the previous period
#     ('lkt', float64), #lower bound for player k
#     ('lmt', float64), #lower bound for player m
#     ('lk_next', float64), #lower bound for k in next period
#     ('lm_next', float64), #lower bound for m in next period
#     ('θk_previous_int', int32), #index 
#     ('lkt_int', int32), #index
#     ('lmt_int', int32), #index
#     ('Pkt', float64), #payment proposal made by k
#     ('Pmt', float64), #payment proposal made by m
#     ('grid_size', int32), #grid size
#     ('W', float64[:,:,:,:,:]) #array with expected values HERE I DON'T KNOW HOW TO ASSIGN DIFFERENT TYPES FOR THE SAME ARRAY
    
# ]


# @jitclass(Bellman_data)
class Bellman:
    #we need to insert 'k = eval(self.creditor)' in every method so it recognizes the creditor correctly
    
        #COMO DEIXAR k e m como variáveis definidas para todas as funções da class?
#     k = eval(self.creditor)


    def __init__(self, creditor, opponent, grid_size = 100):
        self.creditor = creditor
        self.opponent = opponent
        
        #array with expected values
            #θkt
            #ℓkt
            #ℓmt
            #2 slots: responding (0) or proposing (1)
            #period t
        self.W = np.zeros((grid_size, grid_size, grid_size, 2, T))
    
    #liquidation values
    def L(self, t):
        Lst = min(L - C(t), Ds)
        Ljt = min(L - C(t) - Lst, Dj)
        
        if self.creditor == "s":
            return Lst
        if self.creditor == "j":
            return Ljt
    
    #expected continuation value for period t
    #Ex: Ew(t=2) will use θk1 to infer the probabilites for each possible value θk2
    #ℓk2 and ℓm2 will be the lower bounds for t = 2
    def Ew(self, θk_previous, ℓkt, ℓmt, t):
        k = eval(self.creditor)
        
        if(t == T):
            return k.L(t)
        else:
            θk_previous_int =  find(θk_previous)
            ℓkt_int = find(ℓkt)
            ℓmt_int = find(ℓmt)
            
            Ewk_responding = sum(pmf[θk_previous_int, θk_previous_int:100] * k.W[θk_previous_int:100, ℓkt_int, ℓmt_int, 0, (t-1)])
            Ewk_proposing = sum(pmf[θk_previous_int, θk_previous_int:100] * k.W[θk_previous_int:100, ℓkt_int, ℓmt_int, 1, (t-1)])
            
            if self.creditor == "s":
                prob_propose = (1-λj)
            else:
                prob_propose = (λj)
        
            
        return prob_propose * Ewk_proposing + (1-prob_propose) * Ewk_responding
        
    #screening cutoff associated with the payment offer
    def cutoff(self, Pkt, ℓkt, ℓmt, t):
        k = eval(self.creditor)
        m = eval(self.opponent)
       
        #screening cutoff associated with the proposal Pkt
        cmt = ℓmt

        #flag will stop the loop if it takes too long
        flag = 1
        tol = 0.01

        while (Pkt - m.Ew(cmt, cmt, ℓkt, t+1) > tol and flag < grid_size and cmt < 1):
            cmt = cmt + 0.01
            flag = flag + 1
            
        return cmt
    
    
    #payoff when k has the right to offer in t
    def propose(self, θkt, ℓkt, ℓmt, t):
        k = eval(self.creditor)
        m = eval(self.opponent)
        
        #lower bound update
        #it is his ability today, since he reveals it when making an offer
        ℓk_next = θkt
        
        #the optimal offer is the expected continuation value of his opponent next period
        Pkt = m.Ew(expec_beta(ℓmt), ℓmt, ℓk_next, t+1)
        
        cmt = k.cutoff(Pkt, ℓk_next, ℓmt, t)
        ℓm_next = max(ℓmt, cmt)
        
        #Liquidation payoff, reorganization payoff, waiting payoff
        Kt = [k.L(t), V[t] * expec_beta(θkt) - Pkt, k.Ew(θkt, ℓk_next, ℓm_next, t+1)]          

        policy_Kt = np.argmax(Kt)

        #retorns a vector
        return Kt[policy_Kt], policy_Kt, ℓk_next, ℓm_next, Pkt

    
    #payoff and optimal policy when responding to a liquidation offer
    def respond_liq(self, θkt, ℓkt, ℓmt, t):
        k = eval(self.creditor)
        m = eval(self.opponent)
        
        #we only need the expected hability for the next period
        kt_liq = [k.L(t), V[t] * expec_beta(θkt) - m.L(t) ]

        policy_kt_liq = np.argmax(kt_liq)

        return kt_liq[policy_kt_liq], policy_kt_liq

    #probability of a liquidation proposal by his opponent in period t
    #returns a vector with the liquidation probability, ℓk_next, ℓm_next, Pmt
    def prob_liq(self,θkt, ℓkt, ℓmt, t):
        k = eval(self.creditor)
        m = eval(self.opponent)
        
        #this function will be used in the asymetric and symmetric information cases
        
        #in the symmetric information case, the lower bounds will be equal the habiities
        #I used k because it is simpler to populate the matrices latter. I wanted to use "θmt == ℓmt"
        #see ahed how I populate the matrices and it will be clearer
        if(θkt == ℓkt):
            
            #lower bound update
            #ℓmt == θmt in this case
            ℓm_next = ℓmt
            
            #the optimal offer received will be his expected continuation value next period
            Pmt = k.Ew(ℓkt, ℓkt, ℓm_next, t+1)

            ckt = m.cutoff(Pmt, ℓmt, ℓkt, t)
            ℓk_next = max(ℓkt, ckt)
                
            #comparing to the reorganization value
            #it will be just an indicator function, since we know θmt
            if(m.L(t) > V[t] * expec_beta(ℓmt) - Pmt):
                Prob_kt = 1
            else:
                Prob_kt = 0

        #in the asymmetric information case, we will calculate the reorganization threshold
        else:
            
            #lower bound update
            #we don't know what is θmt today, so we use the expected value based on ℓmt as our proxy
            ℓm_next = expec_beta(ℓmt)

            #the optimal offer received will be his expected continuation value next period
            Pmt = k.Ew(expec_beta(ℓkt), ℓkt, ℓm_next, t+1)

            ckt = m.cutoff(Pmt, ℓm_next, ℓkt, t)
            ℓk_next = max(ℓkt, ckt)
            
            if(ℓmt == 1):
                if(m.L(t) > V[t] * expec_beta(ℓmt) - Pmt):
                    Prob_kt = 1
                else:
                    Prob_kt = 0
            else:
                #calculating the threshold
                ϕmt = ℓmt

                #flag will stop the loop if it takes too long
                flag = 1
                tol = 0.01

                while ( (m.L(t) + Pmt)/V[t] - expec_beta(ϕmt) > tol and flag < grid_size and ϕmt < 1):
                    ϕmt = ϕmt + 0.01
                    flag = flag + 1

                #lower bound update
                ℓm_next = ϕmt

                if(ϕmt == ℓmt):
                    #this means that the threshold is lower than ℓmt, so the probability of liquidation is 0
                    #because ϕmt < ℓmt < θmt
                    Prob_kt = 0
                else:
                    #the probability of liquidation will be the CDF(ϕmt|ℓmt)
                    #this is the probability that θmt < ϕmt, given ℓmt
                    Prob_kt = 1 - ((1-ϕmt)**β)/((1 - ℓmt)**β)

        #returns a vector       
        return Prob_kt, ℓk_next, ℓm_next, Pmt
    
#     def threshold(self, θkt, ℓkt, ℓmt, t):
#         k = eval(self.creditor)
#         m = eval(self.opponent)
        
#         ℓm_next = expec_beta(ℓmt)
        
#         Pmt = k.Ew(expec_beta(ℓkt), ℓkt, ℓm_next, t+1)

#         ckt = m.cutoff(Pmt, ℓm_next, ℓkt, t)
#         ℓk_next = max(ℓkt, ckt)


#         #calculating the threshold
#         ϕmt = ℓmt
        

#         #flag will stop the loop if it takes too long
#         flag = 1
#         tol = 0.01

#         while ( (m.L(t) + Pmt)/V[t] - expec_beta(ϕmt) > tol and flag < grid_size and ϕmt < 1):
#             ϕmt = ϕmt + 0.01
#             flag = flag + 1
            
#         return ϕmt
    
    #payoff and optimal policy when responding to a reorganization offer
    #we will use prob_liq to input respond_reorg
    def respond_reorg(self, θkt, ℓk_next, ℓm_next, Pmt, t):
        k = eval(self.creditor)
        m = eval(self.creditor)
        
        kt_reorg = [Pmt, k.Ew(θkt, ℓk_next, ℓm_next,t+1)]
        
        policy_kt_reorg = np.argmax(kt_reorg)
        
        #returns a vector
        return kt_reorg[policy_kt_reorg], policy_kt_reorg
    
    def respond(self, θkt, ℓkt, ℓmt, t):
        k = eval(self.creditor)
        m = eval(self.creditor)
        
        Prob_kt, ℓk_next, ℓm_next, Pmt = k.prob_liq(θkt, ℓkt, ℓmt, t)
        
        kt_respond = Prob_kt * k.respond_liq(θkt, ℓkt, ℓmt, t)[0] + (1-Prob_kt) * k.respond_reorg(θkt, ℓk_next, ℓm_next, Pmt,t)[0]
        
        return kt_respond

    
        
        
    

In [141]:
#versão de testes


#Bellman equations for both players k and his opponent, player m
#only works if we call the creditors "s" or "j" when assigning the Class

# Bellman_data = [
#     ('creditor', nb.types.string), #creditor is as string
#     ('opponent', nb.types.string), #opponent is also a string
#     ('k', nb.types.string), #k is as string
#     ('m', nb.types.string), #m is also a string
#     ('t', int32), #period
#     ('θkt', float64), #real hability of player k
#     ('θk_previous', float64), #real hability of player k in the previous period
#     ('lkt', float64), #lower bound for player k
#     ('lmt', float64), #lower bound for player m
#     ('lk_next', float64), #lower bound for k in next period
#     ('lm_next', float64), #lower bound for m in next period
#     ('θk_previous_int', int32), #index 
#     ('lkt_int', int32), #index
#     ('lmt_int', int32), #index
#     ('Pkt', float64), #payment proposal made by k
#     ('Pmt', float64), #payment proposal made by m
#     ('grid_size', int32), #grid size
#     ('W', float64[:,:,:,:,:]) #array with expected values HERE I DON'T KNOW HOW TO ASSIGN DIFFERENT TYPES FOR THE SAME ARRAY
    
# ]


# @jitclass(Bellman_data)
class Bellman:
    #we need to insert 'k = Bellman(self.creditor, self.opponent)' in every method so it recognizes the creditor correctly
    
        #COMO DEIXAR k e m como variáveis definidas para todas as funções da class?
#     k = Bellman(self.creditor, self.opponent)


    def __init__(self, creditor, opponent, grid_size = 100):
        self.creditor = creditor
        self.opponent = opponent
        
        #array with expected values
            #θkt
            #ℓkt
            #ℓmt
            #2 slots: responding (0) or proposing (1)
            #period t
        self.W = np.zeros((grid_size, grid_size, grid_size, 2, T))
    
    #liquidation values
    def L(self, t):
        Lst = min(L - C(t), Ds)
        Ljt = min(L - C(t) - Lst, Dj)
        
        if self.creditor == "s":
            return Lst
        if self.creditor == "j":
            return Ljt
    
    #expected continuation value for period t
    #Ex: Ew(t=2) will use θk1 to infer the probabilites for each possible value θk2
    #ℓk2 and ℓm2 will be the lower bounds for t = 2
    @njit
    def Ew(self, θk_previous, ℓkt, ℓmt, t):
        k = Bellman(self.creditor, self.opponent)
        
        if(t == T):
            return k.L(t)
        else:
            θk_previous_int =  find(θk_previous)
            ℓkt_int = find(ℓkt)
            ℓmt_int = find(ℓmt)
            
            Ewk_responding = np.sum(pmf[θk_previous_int, θk_previous_int:100] * k.W[θk_previous_int:100, ℓkt_int, ℓmt_int, 0, (t-1)])
            Ewk_proposing = np.sum(pmf[θk_previous_int, θk_previous_int:100] * k.W[θk_previous_int:100, ℓkt_int, ℓmt_int, 1, (t-1)])
            
            if self.creditor == "s":
                prob_propose = (1-λj)
            else:
                prob_propose = (λj)
        
            
        return prob_propose * Ewk_proposing + (1-prob_propose) * Ewk_responding
        
    #screening cutoff associated with the payment offer
    def cutoff(self, Pkt, ℓkt, ℓmt, t):
        k = Bellman(self.creditor, self.opponent)
        m = Bellman(self.opponent,self.creditor)
       
        #screening cutoff associated with the proposal Pkt
        cmt = ℓmt

        #flag will stop the loop if it takes too long
        flag = 1
        tol = 0.01

        while (Pkt - m.Ew(cmt, cmt, ℓkt, t+1) > tol and flag < grid_size and cmt < 1):
            cmt = cmt + 0.01
            flag = flag + 1
            
        return cmt
    
    
    #payoff when k has the right to offer in t
    def propose(self, θkt, ℓkt, ℓmt, t):
        k = Bellman(self.creditor, self.opponent)
        m = Bellman(self.opponent,self.creditor)
        
        #lower bound update
        #it is his ability today, since he reveals it when making an offer
        ℓk_next = θkt
        
        #the optimal offer is the expected continuation value of his opponent next period
        Pkt = m.Ew(expec_beta(ℓmt), ℓmt, ℓk_next, t+1)
        
        cmt = k.cutoff(Pkt, ℓk_next, ℓmt, t)
        ℓm_next = max(ℓmt, cmt)
        
        #Liquidation payoff, reorganization payoff, waiting payoff
        Kt = [k.L(t), V[t] * expec_beta(θkt) - Pkt, k.Ew(θkt, ℓk_next, ℓm_next, t+1)]          

        policy_Kt = np.argmax(Kt)

        #retorns a vector
        return Kt[policy_Kt], policy_Kt, ℓk_next, ℓm_next, Pkt

    
    #payoff and optimal policy when responding to a liquidation offer
    def respond_liq(self, θkt, ℓkt, ℓmt, t):
        k = Bellman(self.creditor, self.opponent)
        m = Bellman(self.opponent,self.creditor)
        
        #we only need the expected hability for the next period
        kt_liq = [k.L(t), V[t] * expec_beta(θkt) - m.L(t) ]

        policy_kt_liq = np.argmax(kt_liq)

        return kt_liq[policy_kt_liq], policy_kt_liq

    #probability of a liquidation proposal by his opponent in period t
    #returns a vector with the liquidation probability, ℓk_next, ℓm_next, Pmt
    def prob_liq(self,θkt, ℓkt, ℓmt, t):
        k = Bellman(self.creditor, self.opponent)
        m = Bellman(self.opponent,self.creditor)
        
        #this function will be used in the asymetric and symmetric information cases
        
        #in the symmetric information case, the lower bounds will be equal the habiities
        #I used k because it is simpler to populate the matrices latter. I wanted to use "θmt == ℓmt"
        #see ahed how I populate the matrices and it will be clearer
        if(θkt == ℓkt):
            
            #lower bound update
            #ℓmt == θmt in this case
            ℓm_next = ℓmt
            
            #the optimal offer received will be his expected continuation value next period
            Pmt = k.Ew(ℓkt, ℓkt, ℓm_next, t+1)

            ckt = m.cutoff(Pmt, ℓmt, ℓkt, t)
            ℓk_next = max(ℓkt, ckt)
                
            #comparing to the reorganization value
            #it will be just an indicator function, since we know θmt
            if(m.L(t) > V[t] * expec_beta(ℓmt) - Pmt):
                Prob_kt = 1
            else:
                Prob_kt = 0

        #in the asymmetric information case, we will calculate the reorganization threshold
        else:
            
            #lower bound update
            #we don't know what is θmt today, so we use the expected value based on ℓmt as our proxy
            ℓm_next = expec_beta(ℓmt)

            #the optimal offer received will be his expected continuation value next period
            Pmt = k.Ew(expec_beta(ℓkt), ℓkt, ℓm_next, t+1)

            ckt = m.cutoff(Pmt, ℓm_next, ℓkt, t)
            ℓk_next = max(ℓkt, ckt)
            
            if(ℓmt == 1):
                if(m.L(t) > V[t] * expec_beta(ℓmt) - Pmt):
                    Prob_kt = 1
                else:
                    Prob_kt = 0
            else:
                #calculating the threshold
                ϕmt = ℓmt

                #flag will stop the loop if it takes too long
                flag = 1
                tol = 0.01

                while ( (m.L(t) + Pmt)/V[t] - expec_beta(ϕmt) > tol and flag < grid_size and ϕmt < 1):
                    ϕmt = ϕmt + 0.01
                    flag = flag + 1

                #lower bound update
                ℓm_next = ϕmt

                if(ϕmt == ℓmt):
                    #this means that the threshold is lower than ℓmt, so the probability of liquidation is 0
                    #because ϕmt < ℓmt < θmt
                    Prob_kt = 0
                else:
                    #the probability of liquidation will be the CDF(ϕmt|ℓmt)
                    #this is the probability that θmt < ϕmt, given ℓmt
                    Prob_kt = 1 - ((1-ϕmt)**β)/((1 - ℓmt)**β)

        #returns a vector       
        return Prob_kt, ℓk_next, ℓm_next, Pmt
    
#     def threshold(self, θkt, ℓkt, ℓmt, t):
#         k = Bellman(self.creditor, self.opponent)
#         m = Bellman(self.opponent,self.creditor)
        
#         ℓm_next = expec_beta(ℓmt)
        
#         Pmt = k.Ew(expec_beta(ℓkt), ℓkt, ℓm_next, t+1)

#         ckt = m.cutoff(Pmt, ℓm_next, ℓkt, t)
#         ℓk_next = max(ℓkt, ckt)


#         #calculating the threshold
#         ϕmt = ℓmt
        

#         #flag will stop the loop if it takes too long
#         flag = 1
#         tol = 0.01

#         while ( (m.L(t) + Pmt)/V[t] - expec_beta(ϕmt) > tol and flag < grid_size and ϕmt < 1):
#             ϕmt = ϕmt + 0.01
#             flag = flag + 1
            
#         return ϕmt
    
    #payoff and optimal policy when responding to a reorganization offer
    #we will use prob_liq to input respond_reorg
    def respond_reorg(self, θkt, ℓk_next, ℓm_next, Pmt, t):
        k = Bellman(self.creditor, self.opponent)
        m = eval(self.creditor)
        
        kt_reorg = [Pmt, k.Ew(θkt, ℓk_next, ℓm_next,t+1)]
        
        policy_kt_reorg = np.argmax(kt_reorg)
        
        #returns a vector
        return kt_reorg[policy_kt_reorg], policy_kt_reorg
    
    def respond(self, θkt, ℓkt, ℓmt, t):
        k = Bellman(self.creditor, self.opponent)
        m = eval(self.creditor)
        
        Prob_kt, ℓk_next, ℓm_next, Pmt = k.prob_liq(θkt, ℓkt, ℓmt, t)
        
        kt_respond = Prob_kt * k.respond_liq(θkt, ℓkt, ℓmt, t)[0] + (1-Prob_kt) * k.respond_reorg(θkt, ℓk_next, ℓm_next, Pmt,t)[0]
        
        return kt_respond

    
        
        
    

Criando as equações de Bellman para os credores

In [142]:
s = Bellman("s", "j")
j = Bellman("j", "s")

In [143]:
#testando as funções

s.propose(0.9, 0.9, 0.9, T-1)

#debuguei prob_liq
s.prob_liq(0.9,0.5,0.5, T-1)

s.respond_liq(0.9,0.9,0.9, T-1)

s.respond_reorg(0.9,0.9,1, 0.10888, T-1)

s.respond(0.9, 0.9, 0.9, T-1)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'Bellman':[0m [1m[1mcannot determine Numba type of <class 'type'>[0m
[1m
File "<ipython-input-141-7f8313a04ddb>", line 65:[0m
[1m    def Ew(self, θk_previous, ℓkt, ℓmt, t):
[1m        k = Bellman(self.creditor, self.opponent)
[0m        [1m^[0m[0m
[0m

This error may have been caused by the following argument(s):
- argument 0: [1mcannot determine Numba type of <class '__main__.Bellman'>[0m


Versão para teste

In [76]:
#https://stackoverflow.com/questions/46696639/numba-jit-nopython-mode-tell-numba-the-signature-of-an-external-arbitrary-funct

#https://www.codespeedy.com/how-to-call-method-of-another-class-in-python/#:~:text=Call%20method%20from%20another%20class,and%20function%20with%20dot%20operator.&text=then%20we%20can%20call%20method_A,%3A%20method_B(self)%3A%20A.
@jit(nopython = True)
def call_eval(x):
    return eval(x)

In [119]:
# Bellman2_data = [
#     ('creditor', nb.types.string), #creditor is as string
#     ('opponent', nb.types.string), #opponent is also a string
#     ('k', nb.types.string), #k is as string
#     ('m', nb.types.string), #m is also a string
#     ('t', int32), #period
#     ('grid_size', int32), #grid size
#     ('W', float64[:,:,:,:,:]) #array with expected values HERE I DON'T KNOW HOW TO ASSIGN DIFFERENT TYPES FOR THE SAME ARRAY
    
# ]


    
# @jitclass(Bellman2_data)
class Bellman2:
    #we need to insert 'k = eval(self.creditor)' in every method so it recognizes the creditor correctly
    
        #COMO DEIXAR k e m como variáveis definidas para todas as funções da class?
#     k = eval(self.creditor)


    def __init__(self, creditor, opponent, grid_size = 100):
        self.creditor = creditor
        self.opponent = opponent
        
        #matrix with expected values
            #θkt
            #ℓkt
            #ℓmt
            #2 slots: responding (0) or proposing (1)
            #period t
        self.W = np.zeros((grid_size, grid_size, grid_size, 2, T))
    
    #liquidation values
    @jit
    def L(self, t):
        Lst = min(L - C(t), Ds)
        Ljt = min(L - C(t) - Lst, Dj)
        
        if self.creditor == "s":
            return Lst
        if self.creditor == "j":
            return Ljt
      
    def diff(self, t):
        k = Bellman(self.creditor, self.opponent)
        m = Bellman(self.opponent,self.creditor)
        
        return k.L(t) - m.L(t)
    
s = Bellman2('s', 'j')
j = Bellman2('j', 's')

s.diff(1)

0.21283000000000002

In [75]:
#tentando usar jit apenas em algumas funções

    
class Bellman3:
    #we need to insert 'k = eval(self.creditor)' in every method so it recognizes the creditor correctly
    
        #COMO DEIXAR k e m como variáveis definidas para todas as funções da class?
#     k = eval(self.creditor)


    def __init__(self, creditor, opponent, grid_size = 100):
        self.creditor = creditor
        self.opponent = opponent
        
        #matrix with expected values
            #θkt
            #ℓkt
            #ℓmt
            #2 slots: responding (0) or proposing (1)
            #period t
        self.W = np.zeros((grid_size, grid_size, grid_size, 2, T))
    
    #liquidation values
    @jit
    def L(self, t):
        Lst = min(L - C(t), Ds)
        Ljt = min(L - C(t) - Lst, Dj)
        
        if self.creditor == "s":
            return Lst
        if self.creditor == "j":
            return Ljt
        
    def diff(self, t):
        k = eval(self.creditor)
        m = eval(self.opponent)
        
#         k = self.creditor
#         m = self.opponent
    
        return k.L(t) - m.L(t)
    
s = Bellman3('s','j')
j = Bellman3('j','s')

s.diff(1)

<ipython-input-75-30307a87435f>:24: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "L" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-75-30307a87435f> (26)

File "<ipython-input-75-30307a87435f>", line 26:
    def L(self, t):
        Lst = min(L - C(t), Ds)
        ^

  @jit
C:\Anaconda3\lib\site-packages\numba\object_mode_passes.py:178: NumbaWarning: Function "L" was compiled in object mode without forceobj=True.

File "<ipython-input-75-30307a87435f>", line 25:
    @jit
    def L(self, t):
    ^

  state.func_ir.loc))
C:\Anaconda3\lib\site-packages\numba\object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-

0.21283000000000002

### Passo 4: populando os arrays

Vamos relembrar a notação de matriz





Populando o array com os valores de liquidação, que independem das habilidades ou lower bounds



In [ ]:
s.W[:,:,:, :,(T-1)] = s.L(T)

j.W[:,:,:, :,(T-1)] = j.L(T)

O que a gente sabe até agora? 

* o valor das funções valor no último período (valor de liquidação)

* calcular as funções valor no penúltimo período, usando St, Jt, st e jt.

Falta apenas a gente popular as matrizes com um loop começando de trás para frente

#### Como é a notação dessa matriz, na prática?


$ W_{st} (\Is{1})   =$ Ws_array[ $\theta_{st}, l_{st}, l_{jt}, t$] 

Exemplo: 

Ws_array[ $0.5, 0.3,  0.2, 1, 14$] = St(0.5, 0.3,0.2,15)

Traduzindo para St:

* St maiúsculo porque a quarta entrada igual a 1 indica que s propõe

* 0.5, 0.3, 0.2 são os mesmos valores dados pelo conjunto de informação de s

* t = 15 em Ws_array porque a entrada 14 do array se refere ao período 15



Loop para popular as matrizes das funções valor no penúltimo período.

É um ensaio antes de rodar o jogo inteiro

In [ ]:
θs_vals
ℓs_vals
ℓj_vals

In [ ]:
#populating the matrices
tempo_total = []

#range para ir do t= T até o t = 1
#só preciso dos valores das matrizes em t=1 porque elas serão usadas para calcular o valor esperado
for t in range(T-1, 0, -1):
    
    

    for θs in θs_vals:
        for ℓs in ℓs_vals:
            for ℓj in ℓj_vals:
                s.W[find(θs), find(ℓs), find(ℓj), 1, (t-1)] = s.propose(θs, ℓs, ℓj, t)[0]
                s.W[find(θs), find(ℓs), find(ℓj), 0, (t-1)] = s.respond(θs, ℓs, ℓj, t)


    for θj in θj_vals:
        for ℓj in ℓj_vals:
            for ℓs in ℓs_vals:
                j.W[find(θj), find(ℓj), find(ℓs), 1, (t-1)] = j.propose(θj, ℓj, ℓs, t)[0]
                j.W[find(θj), find(ℓj), find(ℓs), 0, (t-1)] = j.respond(θj, ℓj, ℓs, t)


    duração = time.process_time() - start
    tempo_total.append(duração)



    print("tempo total para popular o período", t, ":",time.process_time() - start)

In [ ]:
#quanto tempo demora para rodar o jogo com todas as casas decimais em todos os turnos?


sum(tempo_total)/60



##### Salvando os arquivos com os arrays

Só para não ter que ficar gerando os dados toda vez

In [ ]:
#salvando os arrays para poder trabalhar só na parte da negociação depois


np.savetxt('tempo_total_20_08.txt', tempo_total)
np.save('Ws_array_20_08', s.W)
np.save('Wj_array_20_08', j.W)
# np.savetxt('Wj_array.npy', Wj_array)



In [ ]:
#comando para carregar a base
#aqui tem o final .npy
Ws_array = np.load('Ws_array_12_08.npy')

Wj_array = np.load('Wj_array_12_08.npy')

In [ ]:
#teste
# Ws_array[:,:,:,:,0] == Ws_array_load[:,:,:,:,0]

Ws_array[find(0.36),find(0.36),find(0.36),:,0]


Wj_array[find(0.36),find(0.36),find(0.36),:,0]

# Passo 5: resolvendo a negociação entre os credores

Aqui vamos resolver o jogo de fato.

Etapas:

1. Os dois credores recebem suas habilidades, que são informação privada
2. Um credor é sorteado para propor o que fazer com a firma
3. Há uma atualização nos lower bounds das habilidades para o próximo período
4. Usando os lower bounds do próximo período, o credor propositor faz uma oferta baseado no valor de continuação esperado para o próximo período (o dele próprio e o do adversário)
5. Há uma atualização das habilidades verdadeiras para o próximo período, que são informação privada
6. O credor respondente olha a sua habilidade para o próximo período, os lower bounds do próximo período, calcula seu valor de continuação do próximo período e dá uma resposta.
4. O jogo acaba quando os dois concordam quanto ao que fazer com a firma


O resultado final é um vetor contendo 3 coisas:
1. A taxa de recuperação de crédito (o quanto cada credor recebeu em relação ao que ele tinha de crédito)
2. Em qual turno a negociação acabou
3. O que decidiram fazer com a firma (reorganizar ou liquidar)



#### Barganha completa: com s ou j propondo

In [ ]:
#loop while resultado[3] diferente de t, continua



#parâmetros iniciais
t = 0

θst = θ_s0
θjt = θ_j0

#assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
lst = θ_s0
ljt = θ_j0


#vetor com payoff de s, payoff de j, destino da firma e período
#destino da firma é 0 (liquidou) ou 1 (reorganizou)
resultado = np.zeros(4)


#loop roda enquanto não encerrarem o jogo
#o payoff dos jogadores só é gravado no resultado o jogo acaba. 
#Então o loop vai rodar até que o payoff seja diferente do valor inicial, que é zero
while(resultado[0] == 0):
    




    #sorteio do jogador proponente

    u = uniform(0, 1)

    if(u < λj):
        propositor = 'j'

    else:
        propositor = 's'



    if(propositor == 's'):
        



        # se s é chamado a propor ####

        #proposta
        # 0 é liquidar
        # 1 é reorganizar
        # 2 é esperar

        proposta = St(θst, lst, ljt, t)[1]

        payoff_s = St(θst, lst, ljt, t)[0]

        #update dos lower bounds
        ls_next = St(θst, lst, ljt, t)[2]

        #lj_next não pode alterar caso a proposta seja de espera, pois não há cutoff
        lj_next = St(θst, lst, ljt, t)[3]


        #jeito resumido
        # [payoff_s, proposta, ls_next, lj_next] = St(θst, lst, ljt, 0)[1:]

        #update das habilidades à tarde
        θs_next = round(draw_beta(θst), 2)
        θj_next = round(draw_beta(θjt), 2)



        if(proposta == 0):

            #o que j faz se s propor liquidar?
            #j olha sua habilidade e os lower bounds do próximo período e calcula o valor de continuação
            #aqui ele olha o valor de continuação dele de fato, sem ser valor esperado. Então a função é diferente de j_liq

            jt_liq_valor = [Lj(t), V[t] * θj_next - Ls(t) ]

            resposta = np.argmax(jt_liq_valor)

            payoff_j = jt_liq_valor[resposta]


            if(resposta == 0):
                #j concorda em liquidar
                #payoffs são os de liquidação

                resultado[0] = payoff_s
                resultado[1] = payoff_j
                resultado[2] = 0 #0 é caso liquide
                resultado[3] = t


            if(resposta == 1):
                #j prefere reorganizar
                #payoff de s é liq, payoff de j é de reorganização

                resultado[0] = payoff_s
                resultado[1] = payoff_j
                resultado[2] = 1 #1 é caso reorganize
                resultado[3] = t 

            #em qualquer cenário, quando o oponente propôe liquidar o jogo acaba. Por isso sempre temos resultado[3] = t

        if(proposta == 1):
            #o que j faz se s propor reorganizar?

            #a proposta de pagamento de s é o valor esperado da função valor de j, com base nas informações que s tem hoje
            Pst = Ewj(expec_beta(lj_next), ls_next, lj_next, t+1)

            jt_reorg_valor = [Pst, Ewj(θj_next, ls_next, lj_next, t+1)]

            resposta = np.argmax(jt_reorg_valor)

            payoff_j = jt_reorg_valor[resposta]



            if(resposta == 0):
                #j concorda com a proposta de pagamento

                resultado[0] = payoff_s
                resultado[1] = payoff_j
                resultado[2] = 1 #0 é caso liquide
                resultado[3] = t


            if(resposta == 1):
                #j recusa a proposta de pagamento
                #jogo vai para o próximo período
                #vai para o próximo período

                t += 1

                #update das informações
                θst = θs_next
                θjt = θj_next
                lst = ls_next
                ljt = lj_next


        if(proposta == 2):
            #o que acontece se s propor esperar?
            #vai para o próximo período

            t += 1

            #update das informações
            θst = θs_next
            θjt = θj_next
            lst = ls_next
            #ljt não sofre alteração na proposta de esperar


    #============================================================================
    else: #caso quem proponha seja j


        # se j é chamado a propor ####

        #proposta
        # 0 é liquidar
        # 1 é reorganizar
        # 2 é esperar

        proposta = Jt(θst, lst, ljt, t)[1]

        payoff_j = Jt(θst, lst, ljt, t)[0]

        #update dos lower bounds
        ls_next = Jt(θst, lst, ljt, t)[2]

        #ls_next não pode alterar caso a proposta seja de espera, pois não há cutoff
        lj_next = Jt(θst, lst, ljt, t)[3]


        #jeito resumido


        #update das habilidades à tarde
        θs_next = round(draw_beta(θst), 2)
        θj_next = round(draw_beta(θjt), 2)



        if(proposta == 0):

            #o que j faz se s propor liquidar?
            #j olha sua habilidade e os lower bounds do próximo período e calcula o valor de continuação
            #aqui ele olha o valor de continuação dele de fato, sem ser valor esperado. Então a função é diferente de j_liq

            st_liq_valor = [Ls(t), V[t] * θs_next - Lj(t) ]

            resposta = np.argmax(st_liq_valor)

            payoff_s = st_liq_valor[resposta]


            if(resposta == 0):
                #j concorda em liquidar
                #payoffs são os de liquidação

                resultado[0] = payoff_s
                resultado[1] = payoff_j
                resultado[2] = 0 #0 é caso liquide
                resultado[3] = t


            if(resposta == 1):
                #j prefere reorganizar
                #payoff de s é liq, payoff de j é de reorganização

                resultado[0] = payoff_s
                resultado[1] = payoff_j
                resultado[2] = 1 #1 é caso reorganize
                resultado[3] = t 

            #em qualquer cenário, quando o oponente propôe liquidar o jogo acaba. Por isso sempre temos resultado[3] = t

        if(proposta == 1):
            #o que s faz se j propor reorganizar?

            Pjt = Ews(expec_beta(ls_next), ls_next, lj_next, t+1)

            st_reorg_valor = [Pjt, Ews(θs_next, ls_next, lj_next, t+1)]

            resposta = np.argmax(st_reorg_valor)

            payoff_s = st_reorg_valor[resposta]



            if(resposta == 0):
                #s concorda com a proposta de pagamento

                resultado[0] = payoff_s
                resultado[1] = payoff_j
                resultado[2] = 1 #0 é caso liquide
                resultado[3] = t


            if(resposta == 1):
                #s recusa a proposta de pagamento
                #jogo vai para o próximo período

                t += 1

                #update das informações
                θst = θs_next
                θjt = θj_next
                lst = ls_next
                ljt = lj_next


        if(proposta == 2):
            #o que acontece se s propor esperar?
            #vai para o próximo período

            t += 1

            #update das informações
            θst = θs_next
            θjt = θj_next
            ljt = lj_next
            #lst não sofre alteração na proposta de esperar


In [ ]:
θst
θjt

resultado

proposta

t
propositor

#agora falta fazer a média dos valores de negociação para colocar como um momento



### Replicado os gráficos de Dou et. al 
Versão julho de 2020







#### Página 39, Figure 1: Dynamics of Skill and Reorganization Value

In [ ]:
#Panel A:
#período no eixo x, mediana da habilidade de reorganização no eixo y

def median_calc(teta_0, T, ndraws):
    
    #calcula a mediana das habilidades de todos os períodos do jogo
    
    
    
    #matriz onde aparecerão os valores das medianas
    teta_median = np.empty(T, dtype = np.float)
    
    
    
    for t in range(T):
        teta_vals = []
        
        for n in range(ndraws):
            teta = teta_0
            flag = 0
            
            #para garantir que ele tira várias vezes o draw da beta
            while(flag < t):
                teta = draw_beta(teta)
                flag += 1
                
                
            teta_vals.append(teta)
            teta_median[t] = np.median(teta_vals)
    
    return teta_median


#testando
# median_calc(0.28, T, 200)

Median_s = median_calc(θ_s0, 9, 500)
Median_j = median_calc(θ_j0, 9, 500)


In [ ]:
fig, ax = plt.subplots()

#fiz com 9 para ficar igual ao dos autores
X = range(9)



ax.plot(X, Median_j, '-b', linewidth = 3, alpha = 0.8, label = 'Mediana de $θ_{j,t}$')
ax.plot(X, Median_s, '--r', linewidth=3, alpha=1, label = "Mediana de $ θ_{s,t}$")


plt.ylim(0, 1)
plt.xlim(0, 8)

# ax.yaxis.tick_right()
# ax.yaxis.set_ticks_position('both')

#para mostrar ticks dos dois lados do eixo y
plt.tick_params(axis='y', which='both', labelleft='on', labelright='on')



#legenda e labels

plt.xlabel("Period $(t)$")
plt.ylabel("Reorganization skill")

ax.set_title("Réplica de Panel A: Median reorganization skill")
ax.legend(loc = 'upper left')
plt.show()


In [ ]:
#Panel B:
#Valor de reorganização no eixo y, período no eixo x
#aqui precisamos adicionar o valor de reorganização no eixo y. Ele tem o mesmo valor em t = 0 e t = 1


#vamos criar o valor máximao de reorganização usando 9 períodos

V_graf = Vt(Vmax, 9, ρ)


fig, ax = plt.subplots()

#fiz com 9 para ficar igual ao dos autores
X = range(9)



#valor máximo de reorganização de cada um - sem contar os custos
ax.plot(X, V_graf, ':k', linewidth = 3, alpha = 0.8, label = 'Maximum reorg. value $V_t$')
ax.plot(X, Median_j * V_graf, '-b', linewidth = 3, alpha = 0.8, label = 'Junior´s reorg. value $θ_{j,t} V_t$')
ax.plot(X, Median_s * V_graf, '--r', linewidth=3, alpha=1, label = "Senior´s reorg. value $ θ_{s,t} V_t$")


plt.ylim(0, 1)
plt.xlim(0, 8)

# ax.yaxis.tick_right()
# ax.yaxis.set_ticks_position('both')

#para mostrar ticks dos dois lados do eixo y
plt.tick_params(axis='y', which='both', labelleft='on', labelright='on')




plt.xlabel("Period $(t)$")
plt.ylabel("Reorganization value")

ax.set_title("Réplica de Panel B: Median reorganization value")

#achei melhor no upper right
ax.legend(loc = 'upper right')
plt.show()



#### Página 41, Figure 3: Optimal Business Plan

Gráficos da página 41, que mostram o tipo de oferta (liquidar, reorganizar ou esperar) do credor sênior em t = 0. Eixo x é a habilidade verdadeira do credor s e eixo y é o lower bound da habilidade de j.

In [ ]:
#parâmetros da tabela 1, Panel B. 

#peguei as médias dos valores

# #V0/L
# 1.592/0.397

# L_mean = 1/ (1.592/0.397)
# L_mean


# #V0/Dj
# 1.592/0.559

# Dj_mean = 1 / (1.592/0.559)
# Dj_mean

# #Ds
# Ds_mean = Dj_mean/0.559 - Dj_mean
# Ds_mean

In [ ]:
#a proposta de s vem da função St

#o tamanho do grid é igual ao das habilidades

grid_graf = len(θs_vals)


graf1 = np.empty((grid_graf, grid_graf))

for i, θs in enumerate(θs_vals):
    for j, lj in enumerate(lj_vals):
        graf1[i,j] = St(θs, θs, lj, 0)[1]
            
                



In [ ]:
fig, ax = plt.subplots()

cs1 = ax.contourf(θs_vals, lj_vals, graf1.T, alpha=0.75)
# ctr1 = ax.contour(θs_vals, lj_vals, graf1.T)
# plt.clabel(ctr1, inline=1, fontsize=13)
plt.colorbar(cs1, ax = ax)

ax.set_title("Proposta")
ax.set_xlabel("$θ_s$", fontsize=16)
ax.set_ylabel("$l_j$", fontsize=16)

ax.ticklabel_format(useOffset=False)


plt.show()



Debugar o fato de St() não gerar uma função contínua

In [ ]:
#não está contínuo!

for ls in ls_vals:
    St(1, 1, ls, 0)[1]


In [ ]:
#checando porque St(1,1,ls,0) muda quado ls passa de 0.99 para 1

St(1,1,0.99,0)

St(1,1,1,0)

#ok, o payoff muda bastante. Por quê?
#St = [Ls(t), V[t] * θs_next - Pst, Ews(θs_next, ls_next, lj_next, t+1)]
#Pst = Ewj(expec_beta(ljt), ls_next, ljt, t+1)





In [ ]:
#1) caso com ljt = 0.99
Ls(0)
t = 0
ljt = 0.99
Pst = Ewj(expec_beta(ljt), 1, ljt, t+1)

Pst

V[0] * 1 - Pst

Ews(1, 1, 0.99, 1)

In [ ]:
#2) caso com ljt = 1
Ls(0)
t = 0
ljt = 1
Pst = Ewj(expec_beta(ljt), 1, ljt, t+1)

Pst

V[0] * 1 - Pst

Ews(1, 1, 1, 1)

#o valor esperado de j, Ewj(t+1), cai quando ls aumenta... esquisito

In [ ]:
#checando por que Ewj(expec_beta(ljt), 1, ljt, t+1) é maior com ljt = 0.99 do que com ljt = 1

#1) ljt = 0.99


#a expressão é Ewj(expec_beta(ljt), 1, ljt, t+1)
ljt = 0.99
teta_hoje = find(expec_beta(ljt))

pmf[teta_hoje, teta_hoje:100]



#respondendo
Wj_array[teta_hoje:100, find(1), find(0.99), 0, 1 ]


sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100, find(1), find(0.99), 0, 1 ])

#propondo
print("propondo: ", Wj_array[teta_hoje:100, find(1), find(0.99), 1, 1 ])

sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100, find(1), find(0.99), 1, 1 ])

λj * sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100, find(1), find(0.99), 1, 1 ])

(1 - λj) * sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100, find(1), find(0.99), 1, 1 ])

λj * sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100, find(1), find(0.99), 1, 1 ]) + (1 - λj) * sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100, find(1), find(0.99), 1, 1 ])


In [ ]:
#1) ljt = 1

ljt = 1
teta_hoje = find(expec_beta(ljt))

print("pmf:",pmf[teta_hoje, teta_hoje:100])



#respondendo
Wj_array[teta_hoje:100, find(1), find(ljt), 0, 1 ]

    
sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100, find(1), find(ljt), 0, 1 ])

#propondo
print("propondo: ", Wj_array[teta_hoje:100, find(1), find(ljt), 1, 1 ])

sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100, find(1), find(ljt), 1, 1 ])

λj * sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100, find(1), find(ljt), 1, 1 ])

(1 - λj) * sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100, find(1), find(ljt), 1, 1 ])

λj * sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100, find(1), find(ljt), 1, 1 ]) + (1 - λj) * sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100, find(1), find(ljt), 1, 1 ])


#O valor de Wj_array quando j responde e tem ljt igual a 1 não está certo.

In [ ]:
#tínhamos que ter Wj_array[teta_hoje:100, find(1), find(ljt), 0, 1 ] = jt()

# Wj_array[find(i), find(j), find(k),0,(t-1)] = jt(i, j, k, t)


ljt = 1
teta_hoje = find(expec_beta(ljt))

#respondendo
Wj_array[teta_hoje:100, find(1), find(ljt), 0, 1 ]

#deveria ser igual a 

jt(1, 1, 1, 2)


#### Checando se graficamente as funções em t = 1

Primeiro para j

In [ ]:
fig, ax = plt.subplots()

#eixo x, vou colocar do tamanho do lj_vals
X = lj_vals


#guardando os valores das funções no período 1
Ewj_vals = []
Jt_vals = []
jt_vals = []

for ljt in lj_vals:
    Ewj_val = Ewj(expec_beta(ljt), 0.5, ljt, 1)
    Jt_val = Jt(expec_beta(ljt),0.5,ljt,1)[0]
    jt_val = jt(expec_beta(ljt),0.5,ljt,1)
    
    
    Ewj_vals.append(Ewj_val)
    Jt_vals.append(Jt_val)
    jt_vals.append(jt_val)
    
    





#valor máximo de reorganização de cada um - sem contar os custos
ax.plot(X, Ewj_vals, ':k', linewidth = 3, alpha = 0.8, label = 'Ewj')
# ax.plot(X, Jt_vals, '-b', linewidth = 3, alpha = 0.8, label = 'Jt')
# ax.plot(X, jt_vals, '--r', linewidth=3, alpha=1, label = "jt")


# ax.yaxis.tick_right()
# ax.yaxis.set_ticks_position('both')

#para mostrar ticks dos dois lados do eixo y




plt.xlabel("ljt")
plt.ylabel("Valor da função")

ax.set_title("Valor de Ewj")

#achei melhor no upper right
ax.legend(loc = 'lower center')
plt.show()

In [ ]:
#Jt tá muito estranha

fig, ax = plt.subplots()



# ax.plot(X, Ewj_vals, ':k', linewidth = 3, alpha = 0.8, label = 'Ewj')
ax.plot(X, Jt_vals, '-b', linewidth = 3, alpha = 0.8, label = 'Jt')
# ax.plot(X, jt_vals, '--r', linewidth=3, alpha=1, label = "jt")


# ax.yaxis.tick_right()
# ax.yaxis.set_ticks_position('both')

#para mostrar ticks dos dois lados do eixo y




plt.xlabel("ljt")
plt.ylabel("Valor da função")

ax.set_title("Jt")

#achei melhor no upper right
ax.legend(loc = 'lower center')
plt.show()

In [ ]:
Jt(expec_beta(0.6), 1, 0.6, 1)

In [ ]:
#jt


fig, ax = plt.subplots()



# ax.plot(X, Ewj_vals, ':k', linewidth = 3, alpha = 0.8, label = 'Ewj')
# ax.plot(X, Jt_vals, '-b', linewidth = 3, alpha = 0.8, label = 'Jt')
ax.plot(X, jt_vals, '--r', linewidth=3, alpha=1, label = "jt")


# ax.yaxis.tick_right()
# ax.yaxis.set_ticks_position('both')

#para mostrar ticks dos dois lados do eixo y




plt.xlabel("ljt")
plt.ylabel("Valor da função")

ax.set_title("jt")

#achei melhor no upper right
ax.legend(loc = 'lower center')
plt.show()

Agora checando as funções em t = 1 para s

In [ ]:
fig, ax = plt.subplots()

#eixo x, vou colocar do tamanho do lj_vals
X = lj_vals


#guardando os valores das funções no período 1
Ews_vals = []
St_vals = []
st_vals = []

for lst in ls_vals:
    Ews_val = Ews(expec_beta(lst), lst, 0.5, 1)
    St_val = St(expec_beta(lst), lst, 0.5, 1)[0]
    st_val = st(expec_beta(lst), lst, 0.5, 1)
    
    
    Ews_vals.append(Ews_val)
    St_vals.append(St_val)
    st_vals.append(st_val)
    
    





#valor máximo de reorganização de cada um - sem contar os custos
ax.plot(X, Ews_vals, ':k', linewidth = 3, alpha = 0.8, label = 'Ews')
# ax.plot(X, Jt_vals, '-b', linewidth = 3, alpha = 0.8, label = 'Jt')
# ax.plot(X, jt_vals, '--r', linewidth=3, alpha=1, label = "jt")


# ax.yaxis.tick_right()
# ax.yaxis.set_ticks_position('both')

#para mostrar ticks dos dois lados do eixo y




plt.xlabel("lst")
plt.ylabel("Valor da função")

ax.set_title("Ews")

#achei melhor no upper right
ax.legend(loc = 'lower center')
plt.show()

In [ ]:
#St

fig, ax = plt.subplots()



# ax.plot(X, Ewj_vals, ':k', linewidth = 3, alpha = 0.8, label = 'Ewj')
ax.plot(X, St_vals, '-b', linewidth = 3, alpha = 0.8, label = 'St')
# ax.plot(X, jt_vals, '--r', linewidth=3, alpha=1, label = "jt")


# ax.yaxis.tick_right()
# ax.yaxis.set_ticks_position('both')

#para mostrar ticks dos dois lados do eixo y




plt.xlabel("lst")
plt.ylabel("Valor da função")

ax.set_title("St")

#achei melhor no upper right
ax.legend(loc = 'lower center')
plt.show()

In [ ]:
#st


fig, ax = plt.subplots()



# ax.plot(X, Ewj_vals, ':k', linewidth = 3, alpha = 0.8, label = 'Ewj')
# ax.plot(X, Jt_vals, '-b', linewidth = 3, alpha = 0.8, label = 'Jt')
ax.plot(X, st_vals, '--r', linewidth=3, alpha=1, label = "st")


# ax.yaxis.tick_right()
# ax.yaxis.set_ticks_position('both')

#para mostrar ticks dos dois lados do eixo y




plt.xlabel("lst")
plt.ylabel("Valor da função")

ax.set_title("st")

#achei melhor no upper right
ax.legend(loc = 'lower center')
plt.show()

In [ ]:
#payoff de S quanto ele tem o direito de propor em t

 
#Tem que ter preparado o terreno antes calculando as funções valor no período seguinte

def St(teta_st, lst, ljt, t):
    
    #update do lower bound:
    
    #ls_next é a habilidade dele hoje, pois s revela esta quando propõe
    
    ls_next = teta_st
    
      
    
    #quanto s pagará para j neste período? A esperança da função valor de j amanhã,
    #calculadas de acordo com os lower bounds de amanhã
    
    
    #tem expec_beta porque eu não sei o valor de θjt+1, então estimo usando a informação de hoje
    Pst = Ewj(expec_beta(ljt), ls_next, ljt, t+1)
    
    
    #lj_next depende do cutoff de j
    
    #algoritmo para calcular cutoff
    
    cjt = ljt
    
    
    #flag vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    
    #condição cjt < 0.99 é pra ele não passar de 1
    #veja o que acontece com st(0.28, 0.51, 0.28, 14) se não tiver essa condição
    while (Pst - Ewj(cjt, ls_next, cjt, t+1) > tol and flag < 10000 and cjt < 0.99):
        cjt = cjt + 0.01
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    lj_next = max(ljt, cjt)
    
    #aqui calculamos a oferta ótima de novo, pois pode ser que o cutoff seja maior que o lower bound anterior
    #vou deixar sem recalcular. Assim, é como se o pagamento fosse função de ljt
    
#     Pst = Ewj(expec_beta(lj_next), ls_next, lj_next, t+1)
    
    #esperança da própria habilidade no próximo período
    θs_next = expec_beta(teta_st)
    
    #OLHAR ESSA PARTE COM CUIDADO, CONFERIR OS ARGUMENTOS DAS FUNÇÕES NO PERÍODO SEGUINTE
    #conferi em 05/08 e parece OK
    St = [Ls(t), V[t] * θs_next - Pst, Ews(θs_next, ls_next, lj_next, t+1)]
          
    
    policy_St = np.argmax(St)
    
    
    #retorna um vetor: a função valor, a política ótima, o ls_next, e o lj_next
    return St[policy_St],policy_St, ls_next, lj_next



In [ ]:
#checando output completo
St(1,1,0.5,0)

#checando valor esperado do theta_s para amanha
expec_beta(1)

#pegando valores de ls_next, lj_next, e ljt
ljt = 0.5
ls_next = 1
lj_next = 0.54

#checando valor do pagamento
Ewj(expec_beta(ljt), ls_next, ljt, 1) 


#checando valor que faz o threshold funfar
Ewj(lj_next, ls_next, lj_next, 1)

#checando os argumentos de St
Ls(0)

V[0] * expec_beta(1) - Ewj(expec_beta(ljt), ls_next, ljt, 1)

#o valor de continuação dele para o próximo período está dando 0, esquisito isso.
Ews(expec_beta(1), ls_next, lj_next, 1)




Checando Ews(1, 1, 0.54, 1)

In [ ]:
#checando valor de continuação quando teta_s é igual a 1

#valor esperado das funções valor, já organizado

def Ews(teta_hoje, ls_amanha, lj_amanha, t):
    #t é o período para o qual queremos o valor esperado
    #t igual a "t_amanhã"
#     breakpoint()
    if t == T:
        return Ls(T)
    else:
    
        teta_hoje = find(teta_hoje)

        ls_amanha = find(ls_amanha)
        lj_amanha = find(lj_amanha)
    
    
        
        Esp_respondendo = sum(pmf[teta_hoje, teta_hoje:100] * Ws_array[teta_hoje:100,ls_amanha, lj_amanha, 0, (t-1) ])
        Esp_propondo = sum(pmf[teta_hoje, teta_hoje:100] * Ws_array[teta_hoje:100,ls_amanha, lj_amanha, 1, (t-1) ])


        Esp = λj * Esp_respondendo + (1 - λj) * Esp_propondo

        return Esp

In [ ]:


Ews(expec_beta(1), 1, 0.99, 1)

#Esp_respondendo = 0.0


#pmf dá zero
# pmf[teta_hoje, teta_hoje:100]
# array([0.])

#Quando s propõe, o penúltimo argumento é 1. Parece estar OK
# Ws_array[teta_hoje:100,ls_amanha, lj_amanha, 1, (t-1) ]
# array([0.48577883])


#Quando s responde, o penúltimo argumento é 0. Parece ter erro, mesmo que a firma seja liquidada em t=1, s recebe.
# Ws_array[teta_hoje:100,ls_amanha, lj_amanha, 0, (t-1) ]
# array([0.])

#acho que se o resultado da pmf fosse 1, ele multiplicaria por 0.48 e ia ficar sussa

st(1,1,0.54, 2)

In [ ]:
#checando função st()

def st(teta_st, lst, ljt, t):
    
    #agregando todas as funções até agora
#     breakpoint()
    #a função prob_s precisa dos lower bounds do período em questão, então usaremos s_reorg
    [s_reorg_valor, s_reorg_policy, ls_next, lj_next] = s_reorg(teta_st, lst, ljt, t)
    
    Probs = Prob_s(teta_st, ls_next, lj_next, t)
    
    st = (Probs) * s_liq(teta_st, lst, ljt, t)[0] + (1 - Probs) * s_reorg_valor
    
    return (st)

In [ ]:
st(1,1,0.5,1)

In [ ]:
Jt(0.5, 1, 0.5, 1)

In [ ]:
#gráfico B: proposta do credor Júnior em t = 0

graf2 = np.empty((grid_graf, grid_graf))

for i, θs in enumerate(θs_vals):
    for j, lj in enumerate(lj_vals):
        graf2[i,j] = Jt(θs, lj, θs, 0)[1]
        
        


In [ ]:
fig, ax = plt.subplots()

cs1 = ax.contourf(θs_vals, lj_vals, graf2.T, alpha=0.75)
# ctr1 = ax.contour(θs_vals, lj_vals, graf1.T)
# plt.clabel(ctr1, inline=1, fontsize=13)
plt.colorbar(cs1, ax = ax)

ax.set_title("Proposta")
ax.set_xlabel("$θ_j$", fontsize=16)
ax.set_ylabel("$l_s$", fontsize=16)

ax.ticklabel_format(useOffset=False)


plt.show()

# Próximos passos



* HIGIENIZAR O CÓDIGO, DEIXAR ORGANIZADO!


* ~debugar o que acontece quando t = 14~
    * bug na função pmf, e acho que vem da função bins() também


* ~criar função de esperança baseada na função beta~
    * expec_beta(info_hoje) retorna o valor esperado de teta amanhã




* ~discretizar a beta para obter a esperança das funções valor no período seguinte~



* ~fazer Js_val~

* ~guardar resultados de Js_val e Ws_val do último período numa  matriz 4D~

* ~pensar numa função Js_next para achar o valor de Js no próximo período baseado nos parâmetros deste período~
    * regra para atualização de lst
    * regra para atualização de ljt
    * expectativa da habilidade no período seguinte

* ~pensar na forma recursiva do jogo~
    * acho que as funções probabilidade também têm que considerar o update do lower bound, pois elas fazem parte do cenário onde o jogador não propõe


* checar todas as funções de cálculo do jogo (Passo 3)


* considerar a estrutura de negociação do jogo

* aumentar a precisão das estimativas do jogo
    * algoritmo para cálculo do cutoff pode ter mais casas decimais
    
    
 * fazer uma estrutura mais enxuta, com uma função que tome como argumento se é s ou j. Ao invés de criar St, Jt, st, jt...
    


* passos finais
    * replicar os gráficos do artigo de referência


### Conferir novamente

1. ~Se preciso usar ls_next e lj_next nas funções Prob_s e Prob_j~ 
   * conferi e precisa sim, já arrumei

2. Se o valor esperado da função para calcular os thresholds cst e cjt estão corretos

3. A função pmf não está somando um, tem que verificar onde está o erro

